### 1 - SET UP THE API

In [1]:
import requests
import sys
import os
sys.path.insert(0, "/home/admin/ansible/fwd_python_api")
import fwd_json
from fwd_json import fwdApi
username = os.environ['fwd_saas_user']
token = os.environ['fwd_saas_token']
network = 137407
fwd = fwdApi("https://fwd.app/api", username, token,network, {})
nqeUrl = "https://fwd.app/api/nqe?networkId={}".format(network)


In [2]:
#start collection
fwd.start_collection(network).text

'{}'

In [2]:
#get the latest snapshot id 
latest_snap = fwd.get_snapshot_latest(network).json()['id']
print(latest_snap)

470668


In [11]:
#basic NQE to get a report of the network
query = '''
foreach d in network.devices
select {
  name: d.name, 
  mgmtIP: d.platform.managementIps,
  model: d.platform.model,
  osType: d.platform.os,
  osVersion: d.platform.osVersion,
  serial: (foreach c in d.platform.components
           where isPresent(c.serialNumber) select c.serialNumber)
}'''

In [12]:
fwd.post_nqe_check(query)

{'snapshotId': '470668',
 'items': [{'osVersion': '4.20.14M',
   'serial': [],
   'mgmtIP': ['192.168.0.83'],
   'name': 'leaf1',
   'osType': 'ARISTA_EOS',
   'model': 'vEOS'},
  {'osVersion': '4.20.14M',
   'serial': [],
   'mgmtIP': ['192.168.0.84'],
   'name': 'leaf2',
   'osType': 'ARISTA_EOS',
   'model': 'vEOS'},
  {'osVersion': '4.20.14M',
   'serial': [],
   'mgmtIP': ['192.168.0.85'],
   'name': 'leaf3',
   'osType': 'ARISTA_EOS',
   'model': 'vEOS'},
  {'osVersion': '4.20.14M',
   'serial': [],
   'mgmtIP': ['192.168.0.86'],
   'name': 'leaf4',
   'osType': 'ARISTA_EOS',
   'model': 'vEOS'},
  {'osVersion': '4.20.14M',
   'serial': [],
   'mgmtIP': ['192.168.0.81'],
   'name': 'spine1',
   'osType': 'ARISTA_EOS',
   'model': 'vEOS'},
  {'osVersion': '4.20.14M',
   'serial': [],
   'mgmtIP': ['192.168.0.82'],
   'name': 'spine2',
   'osType': 'ARISTA_EOS',
   'model': 'vEOS'}]}

### 2 -  METHOD 1 OF RUNNING NQE: define the query as a string

In [15]:

#define a block 
blockConfig='''
block=```
ip access-list standard BASELINE_ACL
  10   permit 192.168.252.94/31
  20   {"permit" | "deny"} host 192.168.252.221

```;
foreach d in network.devices
where isPresent(d.platform.osVersion)
where d.platform.os == OS.ARISTA_EOS
let diff = blockDiff_alpha1(d.files.config, block)
//where diff.diffCount != 0
select {
  name: d.name, 
  model: d.platform.model,
  missing_config: diff.blocks
}
'''

In [16]:
fwd.post_nqe_check(blockConfig)

{'snapshotId': '470668',
 'items': [{'name': 'leaf1',
   'model': 'vEOS',
   'missing_config': 'ip access-list standard BASELINE_ACL\n  20 {"permit" | "deny"} host 192.168.252.221 **Missing**\n'},
  {'name': 'leaf2',
   'model': 'vEOS',
   'missing_config': 'ip access-list standard BASELINE_ACL\n  20 {"permit" | "deny"} host 192.168.252.221 **Missing**\n'},
  {'name': 'leaf3',
   'model': 'vEOS',
   'missing_config': 'ip access-list standard BASELINE_ACL\n  20 {"permit" | "deny"} host 192.168.252.221 **Missing**\n'},
  {'name': 'leaf4',
   'model': 'vEOS',
   'missing_config': 'ip access-list standard BASELINE_ACL\n  20 {"permit" | "deny"} host 192.168.252.221 **Missing**\n'},
  {'name': 'spine1',
   'model': 'vEOS',
   'missing_config': 'ip access-list standard BASELINE_ACL\n  20 {"permit" | "deny"} host 192.168.252.221 **Missing**\n'},
  {'name': 'spine2',
   'model': 'vEOS',
   'missing_config': 'ip access-list standard BASELINE_ACL\n  20 {"permit" | "deny"} host 192.168.252.221 **M

### 3 -  METHOD 2 OF RUNNING NQE: define the API within NQE

In [17]:
#check baseline CONFIG for all devices 
config = open('baseline_acl.txt', 'r').read()
queryId = "Q_e6ec1965d99271ce3e3a7223897469efc253468f"
payload = {"config": config}
    
response = fwd.post_nqe_para_check(queryId, payload)
missingConfig = response
print(missingConfig)

{'snapshotId': '470668', 'items': [{'name': 'leaf1', 'diff': ''}, {'name': 'leaf2', 'diff': ''}, {'name': 'leaf3', 'diff': ''}, {'name': 'leaf4', 'diff': ''}, {'name': 'spine1', 'diff': ''}, {'name': 'spine2', 'diff': ''}]}


In [3]:
queryId="Q_2f9149a903701261c9477a4808ebc682ecb694cb"
payload= {}
response = fwd.post_nqe_para_check(queryId, payload)
print(response)

{'snapshotId': '470668', 'items': [{'interfaces': '[{teTunnel:null, loopbackMode:false, description:null, aggregation:null, subinterfaces:[{switchedVlans:null, vlan:null, ipv4:{fhrpAddresses:[], addresses:[{ip:192.168.1.1, prefixLength:31}], neighbors:[{ip:192.168.1.0, linkLayerAddress:50:00:00:d7:ee:0b}], fhrp:{hsrp:{fhrpGroups:[]}, vrrp:{fhrpGroups:[]}}}, ipv6:{fhrpAddresses:[], addresses:[], neighbors:[], fhrp:{hsrp:{fhrpGroups:[]}, vrrp:{fhrpGroups:[]}}}, adminStatus:AdminStatus.UP, name:et1, operStatus:OperStatus.UP, networkInstanceName:default, description:null}], mtu:1500, ethernet:{switchedVlan:null, macAddress:50:00:00:d5:5d:c0, aggregateId:null, negotiatedDuplexMode:DuplexMode.FULL, negotiatedPortSpeed:PortSpeed.SPEED_UNKNOWN}, interfaceType:IfaceType.IF_ETHERNET, adminStatus:AdminStatus.UP, operStatus:OperStatus.UP, name:et1, links:[{deviceName:spine1, ifaceName:et1}], bridge:null, tunnel:null, routedVlan:null}, {teTunnel:null, loopbackMode:false, description:null, aggregati

In [19]:
#check BASELINE CONFIG for subset of devices 
config = open('baseline_acl.txt', 'r').read()
queryId = "Q_b7ed24370895b73d6ddbef0b81cffe04d22ae6f5"
#define which device to check
inputDevice = ["leaf1", "spine1", "leaf2"]
payload = {"config": config, "deviceList": inputDevice}
response = fwd.post_nqe_para_check(queryId, payload)
missingConfig = response
print(missingConfig)

{'snapshotId': '470668', 'items': [{'name': 'leaf1', 'diff': ''}, {'name': 'leaf2', 'diff': ''}, {'name': 'spine1', 'diff': ''}]}


In [8]:
#parameterized NQE for BGP neighbor 
queryId = "Q_8178355cfc658ab46cac0f07f2b033f68fa92c80"
payload = {"deviceList": ["leaf4", "leaf2"]}
response = fwd.post_nqe_para_check(queryId, payload)
print(response)

{'snapshotId': '470668', 'items': []}


In [10]:
#parameterized NQE for interfaces that are down 
queryId = "Q_37cac69e9e54556d97b175d8392fa307d7a7afc8"
payload = {"deviceList": ["leaf4"]}
response = fwd.post_nqe_para_check(queryId, payload)
print(response)

{'snapshotId': '470237', 'items': []}


### 4 - PATH SEARCH API

In [28]:
#simple path search api 
srcIP = "192.168.100.1"
dstIP = "192.168.100.4"
fwd.get_path_search(latest_snap,srcIP, dstIP).json()

{'srcIpLocationType': 'INTERFACE',
 'dstIpLocationType': 'INTERFACE',
 'info': {'paths': [{'forwardingOutcome': 'DELIVERED',
    'securityOutcome': 'PERMITTED',
    'hops': [{'deviceName': 'leaf1',
      'deviceType': 'SWITCH',
      'ingressInterface': 'lo0',
      'egressInterface': 'et2',
      'behaviors': ['L3']},
     {'deviceName': 'spine2',
      'deviceType': 'SWITCH',
      'ingressInterface': 'et1',
      'egressInterface': 'et4',
      'behaviors': ['L3']},
     {'deviceName': 'leaf4',
      'deviceType': 'SWITCH',
      'ingressInterface': 'et2',
      'egressInterface': 'self',
      'behaviors': ['L3']}]}],
  'totalHits': {'value': 2, 'type': 'EXACT'}},
 'returnPathInfo': {'paths': [],
  'totalHits': {'value': 0, 'type': 'LOWER_BOUND'}},
 'timedOut': False,
 'queryUrl': 'https://fwd.app/?/go/137407/460422/paths/eyJmcm9tIjp7ImxvY2F0aW9uIjp7InZhbHVlIjoiMTkyLjE2OC4xMDAuMSIsInR5cGUiOiJTdWJuZXRMb2NhdGlvbkZpbHRlciJ9LCJoZWFkZXJzIjpbeyJ2YWx1ZXMiOnsiaXB2NF9kc3QiOlsiMTkyLjE2OC4xMD

In [29]:
#advanced use - define a path search and add as "Existential" intent check

sourceIp = fwd_json.gen_location(SubnetLocationFilter="192.168.100.1/32")
destIp = fwd_json.gen_location(SubnetLocationFilter="192.168.100.4/32")
fwd.post_existance_check(snapshotID=latest_snap, FROM=(sourceIp), TO=(destIp))

*************
{'checkType': 'Existential', 'filters': {'from': {'location': {'type': 'SubnetLocationFilter', 'value': '192.168.100.1/32'}}, 'to': {'location': {'type': 'SubnetLocationFilter', 'value': '192.168.100.4/32'}}, 'mode': 'PERMIT_ALL'}}
*************
{"id":"3847309","definition":{"filters":{"from":{"location":{"value":"192.168.100.1","type":"SubnetLocationFilter"}},"to":{"location":{"value":"192.168.100.4","type":"SubnetLocationFilter"}},"mode":"PERMIT_ALL"},"checkType":"Existential"},"enabled":true,"creationDateMillis":1619546071395,"creatorId":"3416","definitionDateMillis":1619546071395,"status":"PASS","executionDateMillis":1619546071419}


In [30]:
#get results for all "Existential" intent check
result = fwd.get_intent_checks(latest_snap, "Existential").json()
print(result)

/snapshots/460422/checks?type=Existential
[{'id': '3608789', 'definition': {'filters': {'from': {'location': {'value': '192.168.100.2', 'type': 'SubnetLocationFilter'}}, 'to': {'location': {'value': '192.168.100.1', 'type': 'SubnetLocationFilter'}}}, 'noiseTypes': [], 'checkType': 'Existential'}, 'note': 'manually created', 'enabled': True, 'name': 'leaf2 lo100 to db', 'creationDateMillis': 1616077226511, 'creatorId': '3416', 'definitionDateMillis': 1616077226511, 'status': 'PASS', 'executionDateMillis': 1619544127923}, {'id': '3608809', 'definition': {'filters': {'from': {'location': {'value': '192.168.100.3', 'type': 'SubnetLocationFilter'}}, 'to': {'location': {'value': '192.168.100.1', 'type': 'SubnetLocationFilter'}}}, 'noiseTypes': [], 'checkType': 'Existential'}, 'note': 'manually created', 'enabled': True, 'name': 'leaf3 to db', 'creationDateMillis': 1616077241834, 'creatorId': '3416', 'definitionDateMillis': 1616077241834, 'status': 'PASS', 'executionDateMillis': 1619544127929